## HCSIS analysis (+ tutorial)

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# Pandas options
pd.set_option("display.max_columns", 40)
pd.set_option("display.width", 3000)
pd.set_option("display.max_rows", 2000)

In [3]:
# Data location
data_dir = Path("../scraped_data/")
data_file_path = list(data_dir.glob('*.csv'))[0]  # get first csv in dir

In [4]:
# Read CSV
raw = pd.read_csv(data_file_path, dtype={
    'provider_id': 'object', 
    'service_location_id': 'object',
},
parse_dates=['inspection_date'])


In [5]:
# Rearrange cols
raw = raw[['provider_id', 'provider_name', 'service_location', 'service_location_id', 'inspections_found', 'inspection_id','inspection_date','inspection_reason','regulation','non_compliance_area','plans_of_correction','poc_status','certified_locations_url']]


In [6]:
# Basic stats
def unique_vals(df):
    print('UNIQUE VALS:')
    for col in df:
        unique_vals = len(df[col].unique())
        print(f'{col}: {unique_vals}')

n_rows, n_cols = raw.shape
print(f"No. of rows: {n_rows}, No. of cols: {n_cols}\n")
unique_vals(raw)
print()
raw.info()

No. of rows: 44509, No. of cols: 13

UNIQUE VALS:
provider_id: 998
provider_name: 986
service_location: 5955
service_location_id: 899
inspections_found: 3
inspection_id: 17857
inspection_date: 1653
inspection_reason: 7
regulation: 1156
non_compliance_area: 32902
plans_of_correction: 31417
poc_status: 7
certified_locations_url: 998

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44509 entries, 0 to 44508
Data columns (total 13 columns):
provider_id                44509 non-null object
provider_name              44509 non-null object
service_location           44509 non-null object
service_location_id        43928 non-null object
inspections_found          43928 non-null object
inspection_id              43290 non-null object
inspection_date            43290 non-null datetime64[ns]
inspection_reason          43290 non-null object
regulation                 34218 non-null object
non_compliance_area        34218 non-null object
plans_of_correction        34218 non-null object
poc_status

In [7]:
# Date range
earliest_inspection = raw['inspection_date'].min()
most_recent_inspection = raw['inspection_date'].max()
timedelta_between_dates = (most_recent_inspection - earliest_inspection)
days_between_dates = timedelta_between_dates.days
years_between_dates = days_between_dates / 365
total_inspections = len(raw['inspection_id'].unique())
inspections_per_day = total_inspections / days_between_dates
inspections_per_year = total_inspections / years_between_dates

print(f'Earliest inspection: {earliest_inspection}')
print(f'Most recent inspection: {most_recent_inspection}')
print(f'Total number of inspections over period: {total_inspections}')
print(f'Avg. no of inspections per day: {inspections_per_day}')
print(f'Avg. no of inspections per year: {inspections_per_year}')

Earliest inspection: 2010-11-10 00:00:00
Most recent inspection: 2019-10-07 00:00:00
Total number of inspections over period: 17857
Avg. no of inspections per day: 5.489394405164464
Avg. no of inspections per year: 2003.6289578850292


In [8]:
# Create clean copy of df
clean = raw.copy(deep=True)

# Calculate unique service locations
prov_service_loc_id = clean['provider_id'] + '-' + clean['service_location_id']
clean.insert(2,'prov_service_loc_id',prov_service_loc_id)
unique_service_locs = clean['prov_service_loc_id'].unique()
count_unique_service_locs = len(unique_service_locs)
print(f'Unique service locations: {count_unique_service_locs}')

# clean.iloc[500:600].head()

Unique service locations: 6796


In [9]:
# Count of service_locations with no inspection data
locs_no_inspection_data = clean[clean["inspections_found"] == False]
count_locs_no_inspection_data = len(locs_no_inspection_data)
percent_no_inspection_data = (100 / count_unique_service_locs) * count_locs_no_inspection_data
round_percent_no_inspection_data = round(percent_no_inspection_data, 2)

print(f"Service locations with no inspections: {len(locs_no_inspection_data)}")
print(f'% of service locations with no inspections: {round_percent_no_inspection_data}')


Service locations with no inspections: 638
% of service locations with no inspections: 9.39


In [15]:
# interesting violations
df_keyword = clean[clean['non_compliance_area'].str.contains('assault',na=False)]
print(df_keyword['non_compliance_area'].tolist())

["On 4/4/15, sometime between 8:45pm and 8:55pm, Individual #1 who is a 17 year old non-verbal minor male was sexual assaulted by Individual #2, who is a 22 year old adult male.  According to Individual #2's assessment dated 7/25/14, this Individual has a history of inappropriate sexual behaviors, has a habit of infringing on others personal space and requires line of sight supervison until 9pm or asleep.  On 4/4/15, Individual #2 was not supervised and was not asleep at the time of the sexual assult.  ", 'On 09/06/2013 Individual #1 reported to staff #1 that she had been sexually assaulted by staff #2 between July 4th and July 14, 2013. This incident was not reported in HCSIS until September 9, 2013.', 'Individual #1 was both physically & verbally abused by staff #1 on the morning of 3/30/2018.  Staff #1 was yelling and cursing at Individual #1 to get ready to go for bloodwork.  The situation escalated and staff #1 physically assaulted individual #1, scratching her and putting her in 